In [4]:
from langchain.globals import set_debug
set_debug(True)

## CSV Parser

In [5]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.get_format_instructions() # 프롬프트에서 LLM에게 출력 포맷을 instruct 하는 내용

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
llm = ChatGoogleGenerativeAI(model="gemini-pro")

chain = prompt | llm | output_parser
chain.invoke({"subject": "popular Korean cusine"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "subject": "popular Korean cusine"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "subject": "popular Korean cusine"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: List five popular Korean cusine.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] [2.33s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Kimchi, bibimbap, tteokbokki, bulgogi, samgyeopsal",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            

['Kimchi', 'bibimbap', 'tteokbokki', 'bulgogi', 'samgyeopsal']

## JSON Parser

In [8]:
from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
output_parser.get_format_instructions()

'Return a JSON object.'

### 자료구조 정의

In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field

class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")   

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)
output_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'

In [10]:
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template=" Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)
llm = ChatGoogleGenerativeAI(model="gemini-pro")

chain = prompt | llm | output_parser
chain.invoke({"query": "Let me know how to cook Bibimbap"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "query": "Let me know how to cook Bibimbap"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "query": "Let me know how to cook Bibimbap"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human:  Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {\"properties\": {\"foo\": {\"title\": \"Foo\", \"description\": \"a list of strings\", \"type\": \"array\", \"items\": {\"type\": \"string\"}}}, \"required\": [\"foo\"]}\nthe object {\"foo\": [\"bar\", \"baz\"]} is a well-formatted instance of the schema. The object {\"properties\": {\"foo\": [\"bar\", \"baz\"]}} is not well-formatted.\n\nHere is the out

{'name': 'Bibimbap',
 'recipe': '1. Cook rice according to package directions. \n2. While rice is cooking, prepare the vegetables. Thinly slice the carrots, zucchini, and mushrooms. Chop the spinach and green onions. \n3. In a large bowl, combine the cooked rice, vegetables, and meat. \n4. In a small bowl, whisk together the soy sauce, sesame oil, and honey. \n5. Pour the sauce over the rice and vegetables and toss to coat. \n6. Serve immediately, topped with a fried egg if desired.'}